In [1]:
import pandas as pd
#df = pd.read_parquet("../data/EIB2025/doc_descriptions/meterological_data_1.parquet", engine="pyarrow")
import glob
import numpy as np
import csv
import os

file_paths = glob.glob("../data/EIB2025/doc_descriptions/*.parquet")
df = pd.concat([pd.read_parquet(fp) for fp in file_paths], ignore_index=True)

In [2]:
df.head()

,Key Points and Weather Watch,start_date,end_date,Change in Emphasis,Modifications,Evolution and Discussion,Precipitation,Wind,Temperature,Low Cloud / Visibility,Other / Miscellaneous
0,"{'MeterologicalData': ['Low cloud', 'rain', 'h...",2024-04-01 05:35:00,2024-04-02 05:35:00,"{'MeterologicalData': ['sunny spells', 'rain',...","{'MeterologicalData': [], 'MeterologicalDescri...","{'MeterologicalData': ['frontal zones', 'upper...","{'MeterologicalData': [], 'MeterologicalDescri...","{'MeterologicalData': ['breezy', 'windy', 'gus...","{'MeterologicalData': ['maxima', 'frontal clou...","{'MeterologicalData': ['poor BL conditions', '...","{'MeterologicalData': [], 'MeterologicalDescri..."
1,"{'MeterologicalData': ['low cloud', 'rain', 'h...",2024-04-01 08:26:00,2024-04-02 08:26:00,"{'MeterologicalData': ['rain', 'heavy showers'...","{'MeterologicalData': ['Density of showers', '...","{'MeterologicalData': ['frontal zone', 'cold a...","{'MeterologicalData': [], 'MeterologicalDescri...","{'MeterologicalData': ['shower', 'low cloud'],...","{'MeterologicalData': ['maxima', 'frontal clou...","{'MeterologicalData': ['BL conditions', 'ST/co...","{'MeterologicalData': [], 'MeterologicalDescri..."
2,"{'MeterologicalData': ['rain', 'heavy showers/...",2024-04-01 17:42:00,2024-04-02 17:42:00,"{'MeterologicalData': ['heavy showers', 'TS', ...","{'MeterologicalData': ['Density of showers', '...","{'MeterologicalData': ['frontal zone', 'cold a...","{'MeterologicalData': [], 'MeterologicalDescri...","{'MeterologicalData': ['shower', 'low cloud'],...","{'MeterologicalData': ['frost', 'minima', 'max...","{'MeterologicalData': ['poor BL conditions', '...","{'MeterologicalData': [], 'MeterologicalDescri..."
3,"{'MeterologicalData': ['rain', 'low cloud', 'f...",2024-04-01 22:23:00,2024-04-02 22:23:00,"{'MeterologicalData': ['cloudy', 'rain', 'show...","{'MeterologicalData': ['shower intensities'], ...","{'MeterologicalData': ['frontal zone', 'cold a...","{'MeterologicalData': [], 'MeterologicalDescri...","{'MeterologicalData': ['shower', 'low cloud'],...","{'MeterologicalData': ['frost', 'minima', 'max...","{'MeterologicalData': ['poor BL conditions', '...","{'MeterologicalData': [], 'MeterologicalDescri..."
4,"{'MeterologicalData': ['rain', 'low cloud', 'f...",2024-04-02 03:38:00,2024-04-03 03:38:00,"{'MeterologicalData': ['Rain', 'low cloud', 'h...","{'MeterologicalData': ['heavier showers', 'sho...","{'MeterologicalData': ['low pressure systems',...","{'MeterologicalData': ['showers', 'heavier sho...","{'MeterologicalData': ['windier', 'near gales'...","{'MeterologicalData': ['minima', 'maxima', 'fr...","{'MeterologicalData': ['poor BL conditions', '...","{'MeterologicalData': [], 'MeterologicalDescri..."


In [3]:
file_paths

['../data/EIB2025/doc_descriptions/meterological_data_4.parquet',
 '../data/EIB2025/doc_descriptions/meterological_data_1.parquet',
 '../data/EIB2025/doc_descriptions/meterological_data_2.parquet',
 '../data/EIB2025/doc_descriptions/meterological_data_3.parquet']

In [4]:
data_dir = "EIB2025/"
csv_path = "../data/EIB2025/meteo_data_paths.csv"
sub_string ="C:\\PERSONAL\\UK PHD\\2025_eib_hackerton\\"

npy_paths = []
dates = []

# i have to index csv to drop dates and add descriptions and labels
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        npy_paths.append(row["path"])
        dates.append(row["date"])

'changed to linux paths'
npy_paths = [os.path.join(data_dir, x.replace(sub_string, "").replace("\\", "/")) for x in npy_paths]

In [5]:
met_dates = [str(date) for date in df['start_date'].to_list()]

In [6]:
print(len(npy_paths))
print(len(met_dates))

447
465


In [7]:
from datetime import datetime

def get_time_bin(time_str):
    """Converts a time string to its corresponding 6-hour bin."""
    time_obj = datetime.strptime(time_str.strip(), "%H:%M:%S")
    hour = time_obj.hour

    if 0 <= hour < 6:
        return '_000000'
    elif 6 <= hour < 12:
        return '_060000'
    elif 12 <= hour < 18:
        return '_120000'
    else:
        return '_180000'

# bin the dates for given 6 hour time range
binned_dates = [date[:10]+get_time_bin(date[10:]) for date in met_dates]
df['binned_date_times'] = binned_dates # add to data frame to index 

cleaned_npy_dates = [date for date in dates if date in binned_dates] # retrieve date-times that we have data for
cleaned_met_dates = [date for date in binned_dates if date in cleaned_npy_dates] # sanity check

print(len(cleaned_npy_dates))
print(len(cleaned_met_dates))

430
430


In [8]:
cleaned_npy_data = {}
cleaned_npy_p = []

for i, date in enumerate(dates):
    if date in binned_dates:
        cleaned_npy_data[date] = {'path': npy_paths[i]}

In [9]:
 df["Key Points and Weather Watch"][0] # need description and labels

{'MeterologicalData': array(['Low cloud', 'rain', 'heavy showers'], dtype=object),
 'MeterologicalDescription': array(['Low cloud now extensive across central and E England, as well as for North Sea coastal areas',
        'rain over parts of NE England and SE Scotland on Monday',
        'heavy showers/TS in post-frontal airmass S/SW England, S Wales and S Midlands Monday afternoon'],
       dtype=object)}

In [13]:
# collect descritptions and labels, added to correct date
for i, date in enumerate(cleaned_npy_data.keys()):
    row_values = df[df['binned_date_times'] == date]['Key Points and Weather Watch'].to_dict()
    row_values = list(row_values.values())[0]
    info = {'label': row_values['MeterologicalData'],
           'description': ' '.join(row_values['MeterologicalDescription'])}
    cleaned_npy_data[date].update(info)

In [14]:
# Example of data point with empty labels and description
row_values = df[df['binned_date_times'] == '2024-01-06_060000']['Key Points and Weather Watch'].to_dict()
row_values

{136: {'MeterologicalData': array([], dtype=object),
  'MeterologicalDescription': array([], dtype=object)}}

In [15]:
len(cleaned_npy_data.keys())

430

In [16]:
final_cleaned_npy_data = {k: v for k, v in cleaned_npy_data.items() if len(v['label']) and v['description']}
len(final_cleaned_npy_data.keys())

428

In [17]:
flattened = []
for key, value in final_cleaned_npy_data.items():
    flattened.append({
        'datetime': key,
        'path': value['path'],
        'label': ' '.join(value['label'].tolist()),
        'description': value['description']
    })

final_dataset = pd.DataFrame(flattened)
final_dataset.to_csv('dataset.csv', index=False)

In [18]:
final_dataset.head()

,datetime,path,label,description
0,2024-01-01_000000,EIB2025/npy_files_2024_single_timesteps/2024-0...,wet weather windy,spells of wet weather following across much of...
1,2024-01-01_060000,EIB2025/npy_files_2024_single_timesteps/2024-0...,wet weather windy,spells of wet weather across many parts later ...
2,2024-01-01_120000,EIB2025/npy_files_2024_single_timesteps/2024-0...,wet weather windy,spells of wet weather across many parts later ...
3,2024-01-01_180000,EIB2025/npy_files_2024_single_timesteps/2024-0...,wet weather windy,spells of wet weather across many parts tonigh...
4,2024-01-02_000000,EIB2025/npy_files_2024_single_timesteps/2024-0...,wet weather windy,spells of wet weather across many parts during...


In [19]:
final_cleaned_npy_data['2024-01-08_060000']

{'path': 'EIB2025/npy_files_2024_single_timesteps/2024-01-08_060000.npy',
 'label': array(['colder conditions', 'drier conditions', 'quieter conditions',
        'cloudier weather', 'fog', 'ice', 'showers', 'snow showers'],
       dtype=object),
 'description': 'much colder conditions now established much drier conditions now established much quieter conditions now established cloudier slightly less cold weather probably spreading slowly S’wards later some fog given recent wet weather ice given recent wet weather showers running into E of NI and SE Scotland snow showers running W’wards across S areas today and first half of tonight'}